In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install llmx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 26.5 MB/s eta 0:00:00


In [ ]:
!pip install cohere


In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00


In [ ]:
import os
import openai
import random
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from transformers import pipeline


In [ ]:
openai.api_key = "OPENAI_API_KEY"


In [ ]:

prompt = "A model that takes take an event name and can generate the food items suggestions for that event, either readymade or recipes. Dont generate the food description, just generate the list of items."
temperature = .4
number_of_examples = 20

In [ ]:
def generate_example(prompt, prev_examples, temperature=.5):
    messages=[
        {
            "role": "system",
            "content": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.Dont generate the description of food items, just give a list of food items relevant to the prompt.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`"
        }
    ]

    if len(prev_examples) > 0:
        if len(prev_examples) > 10:
            prev_examples = random.sample(prev_examples, 10)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        temperature=temperature,
        max_tokens=1354,
    )

    return response.choices[0].message['content']

# Generate examples
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)

print(prev_examples)

Generating example 0
Generating example 1
Generating example 2
Generating example 3
Generating example 4
Generating example 5
Generating example 6
Generating example 7
Generating example 8
Generating example 9
Generating example 10
Generating example 11
Generating example 12
Generating example 13
Generating example 14
Generating example 15
Generating example 16
Generating example 17
Generating example 18
Generating example 19
["prompt\n-----------\nSuggest some food items for a children's birthday party.\n-----------\nresponse\n-----------\n1. Mini Pizzas\n2. Chicken Nuggets\n3. Chocolate Cupcakes\n4. Fruit Skewers\n5. Jelly\n6. Cheese Sandwiches\n7. Popcorn\n8. Ice Cream Sundae\n9. Candy Floss\n10. Vegetable Sticks with Hummus\n-----------\nprompt\n-----------\nWhat food items would be suitable for a vegan potluck?\n-----------\nresponse\n-----------\n1. Vegan Lasagna\n2. Quinoa Salad\n3. Vegan Sushi Rolls\n4. Chickpea Curry\n5. Vegan Tacos\n6. Lentil Soup\n7. Vegan Chocolate Cake\n8.

In [ ]:
def generate_system_message(prompt):

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
          {
            "role": "system",
            "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "content": prompt.strip(),
          }
        ],
        temperature=temperature,
        max_tokens=500,
    )

    return response.choices[0].message['content']

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

The system message is: `Given an event name, generate a list of food item suggestions suitable for that event.`. Feel free to re-run this cell if you want a better result.


In [ ]:
import pandas as pd

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
  try:
    split_example = example.split('-----------')
    prompts.append(split_example[1].strip())
    responses.append(split_example[3].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples. Here are the first few:')

df.head()


There are 20 successfully-generated examples. Here are the first few:


,prompt,response
0,Suggest some food items for a children's birth...,1. Mini Pizzas\n2. Chicken Nuggets\n3. Chocola...
1,What food items should I prepare for a traditi...,1. Roasted Turkey\n2. Stuffing\n3. Mashed Pota...
2,Suggest some food items for a Diwali party.,1. Samosas\n2. Butter Chicken\n3. Paneer Tikka...
3,Suggest some food items for a St. Patrick's Da...,1. Corned Beef and Cabbage\n2. Irish Soda Brea...
4,Suggest some food items for a Ramadan Iftar meal.,1. Dates\n2. Lentil Soup\n3. Samosas\n4. Chick...


In [ ]:
# Split the data into train and test sets, with 90% in the train set
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

# Save the dataframes to .jsonl files
train_df.to_json('/content/drive/MyDrive/grocer_ease2/train.jsonl', orient='records', lines=True)
test_df.to_json('/content/drive/MyDrive/grocer_ease2/test.jsonl', orient='records', lines=True)

NameError: ignored

In [ ]:
_# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "/content/drive/MyDrive/grocer_ease2/train.jsonl"

# Fine-tuned model name
new_model = "llama-2-7b-finetuned"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "/content/drive/MyDrive/grocer_ease2/results"

# Number of training epochs
num_train_epochs = 4

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 2

# Batch size per GPU for evaluation
per_device_eval_batch_size = 2

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load datasets
train_dataset = load_dataset('json', data_files='/content/drive/MyDrive/grocer_ease2/train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='/content/drive/MyDrive/grocer_ease2/test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

{'loss': 1.4388, 'learning_rate': 4.735678371226441e-05, 'epoch': 2.78}
{'train_runtime': 17.137, 'train_samples_per_second': 4.201, 'train_steps_per_second': 2.101, 'train_loss': 1.2214988867441814, 'epoch': 4.0}


In [ ]:
# Cell 4: Test the model
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n what should I prepare for halloween [/INST]" # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])

[INST] <<SYS>>
Given an event name, generate a list of food item suggestions suitable for that event.
<</SYS>>

 what should I prepare for halloween [/INST]  Great! Here are some food item suggestions that are perfect for a spooky Halloween party:

1. Pumpkin-themed treats: Pumpkin pie, pumpkin bread, pumpkin bread pudding, pumpkin spice latte, pumpkin-shaped cookies, pumpkin-themed cupcakes.
2. Spooky snacks: Ghostly deviled eggs, witches' brew (a green-colored punch), spider web (a cheese and crackers platter), monster eyes (deviled eggs with black olives), bats (chocolate-dipped strawberries).
3. Creepy


In [ ]:
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\nWrite the recipes and foods associated with the event diwali. [/INST]" # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])

[INST] <<SYS>>
Given an event name, generate a list of food item suggestions suitable for that event.
<</SYS>>

Write the recipes and foods associated with the event diwali. [/INST]  Sure, here are some traditional food suggestions for Diwali:

1. Sweets:
	* Gulab Jamun
	* Jalebi
	* Barfi
	* Ladoo
	* Mysore Pak
	* Petha
	* Chakli
	* Dhokla
	* Kheer
	* Payasam
2. Savory Snacks:
	* Samosas
	* Chaat
	* Papdi Chaat
	* Dahi Vada
	* Chole Bhature
	* Aloo Tikki
	* Kachoris
	* Chivda
	* Bh


In [ ]:

prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\nGive the relevant foods associated with holi. [/INST]" # replace the command here with something relevant to your task
num_new_tokens = 500  # change to the number of new tokens you want to generate

# Count the number of tokens in the prompt
num_prompt_tokens = len(tokenizer(prompt)['input_ids'])

# Calculate the maximum length for the generation
max_length = num_prompt_tokens + num_new_tokens

gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
result = gen(prompt)
print(result[0]['generated_text'].replace(prompt, ''))

  Sure, here are some traditional Indian food items that are commonly associated with the Hindu festival of Holi:

1. Gujiyas - a sweet, fried doughnut-like snack that is typically served during Holi.
2. Thandai - a cold drink made with milk, almonds, and spices, often served during Holi to cool down from the heat of the festival.
3. Pani Puri - a popular street food in India, pani puri is a snack made with crispy bread, spicy water, and a variety of toppings such as potatoes, onions, and chickpeas.
4. Mathri - a crunchy, savory snack made with wheat flour, water, and spices, often served during Holi.
5. Gajar Ka Halwa - a sweet, slow-cooked carrot pudding that is a popular dish during Holi.
6. Jalebi - a sweet, deep-fried doughnut that is often served during Holi.
7. Kachoris - a crispy, savory snack made with wheat flour, water, and spices, often served during Holi.
8. Chole Bhature - a spicy chickpea curry served with a deep-fried bread called bhature, which is often enjoyed during 

In [ ]:
# Empty VRAM
# del model
# del pipe
# del trainer
import gc
gc.collect()
gc.collect()

0

In [ ]:
# Merge and save the fine-tuned model
from google.colab import drive
# drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/grocer_ease2/model"

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Save the merged model
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: ignored

In [ ]:
# drive.mount('/content/drive')

model_path = "/content/llama-2-7b-custom"  # change to your preferred path

# Save the merged model
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('/content/llama-2-7b-custom/tokenizer_config.json',
 '/content/llama-2-7b-custom/special_tokens_map.json',
 '/content/llama-2-7b-custom/tokenizer.json')

In [ ]:

# from google.colab import drive
# from transformers import AutoModelForCausalLM, AutoTokenizer

# drive.mount('/content/drive')

model_path = "/content/llama-2-7b-custom"  # change to the path where model is saved

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:

from transformers import pipeline

prompt = "Write the recipes and foods associated with the event diwali"  # change to your desired prompt
gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length = 200)
result = gen(prompt)
print(result)

[{'generated_text': 'Write the recipes and foods associated with the event diwali.\nDiwali, the festival of lights, is a significant event in Hinduism and is celebrated with great enthusiasm and joy across the world. It is a five-day festival that starts on the 13th day of the Hindu month of Ashvina and ends on the 18th day of the month of Kartika. During this festival, people clean their homes, decorate them with diyas (small clay lamps), and exchange gifts and sweets with their loved ones.\nHere are some recipes and foods associated with Diwali:\n1. Sweets: Diwali is a time for indulging in delicious sweets and savouries. Some popular sweets include laddu (a ball-shaped sweet made with flour, sugar, and ghee), barfi (a dense, sweet dessert'}]


In [ ]:

import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:

!huggingface-cli login

# model.push_to_hub("MoinFaisal/llama-2-7b-custom", check_pr=True)

# tokenizer.push_to_hub("MoinFaisal/llama-2-7b-custom",check_pr=True)



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

In [ ]:
model.push_to_hub("aishwarya21/llama-2-7b-grocer_ease", check_pr=True)

tokenizer.push_to_hub("aishwarya21/llama-2-7b-grocer-ease",check_pr=True)

KeyboardInterrupt: ignored